In [2]:
# Setup
import boto3, sagemaker, os
from sagemaker.sklearn import SKLearn
from sagemaker import get_execution_role

region = "us-east-1"
bucket = "grey-customer-feedback-bucket" 
prefix = "training"

session = sagemaker.Session()
role = get_execution_role()

# Upload prepared data to S3
train_s3 = session.upload_data(path="data/train_data.csv", bucket=bucket, key_prefix=f"{prefix}/data")
val_s3   = session.upload_data(path="data/val.csv", bucket=bucket, key_prefix=f"{prefix}/data")

train_s3, val_s3

('s3://grey-customer-feedback-bucket/training/data/train_data.csv',
 's3://grey-customer-feedback-bucket/training/data/val.csv')

In [ ]:
  %%writefile train.py # Cell 2 I created train.py

In [10]:
# Define Estimator & Train
output_path = "s3://grey-customer-feedback-bucket/models/"

estimator = SKLearn(
    entry_point="train.py",
    role=role,
    instance_type="ml.m5.large",
    instance_count=1,
    framework_version="1.2-1",
    py_version="py3",
    base_job_name="feedback-analyzer-train",
    output_path=output_path
)

estimator.fit({
    "train": train_s3,
    "val": val_s3
})


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: feedback-analyzer-train-2025-08-13-20-43-32-989


2025-08-13 20:43:36 Starting - Starting the training job...
2025-08-13 20:43:51 Starting - Preparing the instances for training...
2025-08-13 20:44:37 Downloading - Downloading the training image........2025-08-13 20:45:42,117 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-08-13 20:45:42,120 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-13 20:45:42,122 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-08-13 20:45:42,138 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-08-13 20:45:42,461 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-13 20:45:42,465 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-08-13 20:45:42,483 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-13 20:45:42,486 sagemaker-train

In [ ]:
# Deploy Endpoint
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",  # cheap real-time endpoint
    endpoint_name="feedback-analyzer-endpoint"
)

endpoint_name = predictor.endpoint_name
print(f"✅ Endpoint deployed: {endpoint_name}")

INFO:sagemaker:Creating model with name: feedback-analyzer-train-2025-08-13-20-55-51-892
INFO:sagemaker:Creating endpoint-config with name feedback-analyzer-endpoint
INFO:sagemaker:Creating endpoint with name feedback-analyzer-endpoint


-----------------------------------------------